Wine.com has not responded about upgrading my API access and I feel it would be rude to scrape this early in the process, so I will do some algorithm testing by using a dataset from the UC Irvine Machine Learning Repository.

Link to dataset:  
https://archive.ics.uci.edu/ml/datasets/Wine+Quality

Link to original paper:  
http://www.sciencedirect.com/science/article/pii/S0167923609001377

Citation:  
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.   
  Modeling wine preferences by data mining from physicochemical properties.  
  In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.  

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt

from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

//anaconda/lib/python3.5/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [3]:
with open('winequality-red.csv') as file:
    red_data = pd.read_csv(file, delimiter=';')
red_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5
6,7.9,0.600,0.06,1.6,0.069,15.0,59.0,0.99640,3.30,0.46,9.4,5
7,7.3,0.650,0.00,1.2,0.065,15.0,21.0,0.99460,3.39,0.47,10.0,7
8,7.8,0.580,0.02,2.0,0.073,9.0,18.0,0.99680,3.36,0.57,9.5,7
9,7.5,0.500,0.36,6.1,0.071,17.0,102.0,0.99780,3.35,0.80,10.5,5


In [4]:
with open('winequality-white.csv') as file:
    white_data = pd.read_csv(file, delimiter=';')
white_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.270,0.36,20.70,0.045,45.0,170.0,1.00100,3.00,0.45,8.800000,6
1,6.3,0.300,0.34,1.60,0.049,14.0,132.0,0.99400,3.30,0.49,9.500000,6
2,8.1,0.280,0.40,6.90,0.050,30.0,97.0,0.99510,3.26,0.44,10.100000,6
3,7.2,0.230,0.32,8.50,0.058,47.0,186.0,0.99560,3.19,0.40,9.900000,6
4,7.2,0.230,0.32,8.50,0.058,47.0,186.0,0.99560,3.19,0.40,9.900000,6
5,8.1,0.280,0.40,6.90,0.050,30.0,97.0,0.99510,3.26,0.44,10.100000,6
6,6.2,0.320,0.16,7.00,0.045,30.0,136.0,0.99490,3.18,0.47,9.600000,6
7,7.0,0.270,0.36,20.70,0.045,45.0,170.0,1.00100,3.00,0.45,8.800000,6
8,6.3,0.300,0.34,1.60,0.049,14.0,132.0,0.99400,3.30,0.49,9.500000,6
9,8.1,0.220,0.43,1.50,0.044,28.0,129.0,0.99380,3.22,0.45,11.000000,6


In [5]:
red_data['Is Red?'] = 1
white_data['Is Red?'] = 0

In [6]:
data = pd.concat([red_data, white_data], ignore_index=True)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Is Red?
0,7.4,0.700,0.00,1.90,0.076,11.0,34.0,0.99780,3.51,0.56,9.400000,5,1
1,7.8,0.880,0.00,2.60,0.098,25.0,67.0,0.99680,3.20,0.68,9.800000,5,1
2,7.8,0.760,0.04,2.30,0.092,15.0,54.0,0.99700,3.26,0.65,9.800000,5,1
3,11.2,0.280,0.56,1.90,0.075,17.0,60.0,0.99800,3.16,0.58,9.800000,6,1
4,7.4,0.700,0.00,1.90,0.076,11.0,34.0,0.99780,3.51,0.56,9.400000,5,1
5,7.4,0.660,0.00,1.80,0.075,13.0,40.0,0.99780,3.51,0.56,9.400000,5,1
6,7.9,0.600,0.06,1.60,0.069,15.0,59.0,0.99640,3.30,0.46,9.400000,5,1
7,7.3,0.650,0.00,1.20,0.065,15.0,21.0,0.99460,3.39,0.47,10.000000,7,1
8,7.8,0.580,0.02,2.00,0.073,9.0,18.0,0.99680,3.36,0.57,9.500000,7,1
9,7.5,0.500,0.36,6.10,0.071,17.0,102.0,0.99780,3.35,0.80,10.500000,5,1


**More on the features**
* <a href="http://waterhouse.ucdavis.edu/whats-in-wine/fixed-acidity">fixed acidity</a>
    * in general, tartaric, malic, citric, and succinic acids
    * only measuring tartaric here
    * Units: (g(tartaric acid)/dm3)
* <a href="http://waterhouse.ucdavis.edu/whats-in-wine/volatile-acidity">volatile acidity</a>
    * mostly acetic acid, but also lactic, formic, butyric, and propionic acids.
    * not usually desirable
    * Units: (g(acetic acid)/dm3)
* citric acid
    * a type of fixed acid
    * Units: (g/dm3)
* residual sugar
    * Units: (g/dm3)
* chlorides
    * Since this is measuring NaCl, it is a measure of saltiness
    * Units: (g(sodium chloride)/dm3)
* free sulfur dioxide
    * Units: (mg/dm3)
* total sulfur dioxide
    * Units: (mg/dm3)
* density
    * Units: (g/cm3)
* pH
* sulphates
    * This doesn't seem to have anything to do with wine, so I'm not sure why they tested for it
    * It had an impact in the results of the paper, though...
    * Units: (g(potassium sulphate)/dm3)
* alcohol
    * Units: (vol.%)
    
**About sulfur dioxide**
A quick chemcial aside: sulfur dioxide is the compound in question, but it is not technically a sulfite. Most people just call it that. Sulfur dioxide is mostly used as a preservative. In high enough concentrations, it could affect the taste, but not usually. Sulfur dioxide can bind with aldehydes, which neutralizes the aroma of the aldehyde. Molecules that aren't bound are called free.
Links:  
http://waterhouse.ucdavis.edu/whats-in-wine/sulfites-in-wine
http://www.aromadictionary.com/articles/sulfurdioxide_article.html

**Absent features**
* <a href="http://waterhouse.ucdavis.edu/whats-in-wine/oak-lactones">beta-methyl-gamma-octalactones</a>
    * the compound responsible for flavor from oaking
* malolactic fermentation
    * the volatile acidity could be an imperfect proxy for this
        * malolactic fermentation produces volatile acids, but so does alcoholic fermentation

**Paper on recommender systems**  
http://josquin.cs.depaul.edu/~rburke/pubs/burke-umuai02.pdf

The dataset has only one column for quality, so I'm going to try to synthesize some data to work with. I'm going to use K-means clustering to generate some clusters based on all features but 'quality' and 'Is Red?' and create some users that have preferences for different clusters.

In [7]:
model = KMeans(n_clusters=4, random_state=5)

In [8]:
data.values[:,0:-3].shape

(6497, 10)

In [9]:
model.fit(data.values[:,0:-3])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=5, tol=0.0001, verbose=0)

In [10]:
model.cluster_centers_

array([[  6.97070064e+00,   2.94322111e-01,   3.53976342e-01,
          9.33525933e+00,   5.18798908e-02,   5.08430391e+01,
          1.97769336e+02,   9.96285473e-01,   3.18171065e+00,
          5.14840764e-01],
       [  6.95822352e+00,   3.12706198e-01,   3.13523670e-01,
          4.10795510e+00,   4.84387506e-02,   2.52144949e+01,
          9.87227916e+01,   9.93077994e-01,   3.20698389e+00,
          5.06730112e-01],
       [  6.89117350e+00,   2.82612544e-01,   3.37440567e-01,
          6.72895802e+00,   4.83019727e-02,   3.71383409e+01,
          1.44657562e+02,   9.94440432e-01,   3.19498735e+00,
          4.91876581e-01],
       [  8.26224490e+00,   4.98462099e-01,   2.70852770e-01,
          2.46712828e+00,   8.18454810e-02,   1.26526968e+01,
          3.37995627e+01,   9.96210459e-01,   3.29905248e+00,
          6.37835277e-01]])

In [11]:
labels = pd.DataFrame(model.labels_, columns=['Cluster'])
labels.head()

,Cluster
0,3
1,1
2,3
3,3
4,3


In [12]:
cluster_data = pd.concat([data,labels], axis=1)
cluster_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Is Red?,Cluster
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1,3
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1,3
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1,3


In [13]:
cluster_groups = cluster_data.groupby('Cluster')

In [14]:
cluster_groups.describe()

Is Red?      alcohol    chlorides  citric acid  \
Cluster                                                             
0       count  1100.000000  1100.000000  1100.000000  1100.000000   
        mean      0.001818     9.712227     0.051879     0.353845   
        std       0.042621     0.825674     0.025701     0.126787   
        min       0.000000     8.500000     0.019000     0.000000   
        25%       0.000000     9.100000     0.042000     0.270000   
        50%       0.000000     9.500000     0.048000     0.330000   
        75%       0.000000    10.000000     0.054000     0.420000   
        max       1.000000    13.300000     0.346000     1.230000   
1       count  2047.000000  2047.000000  2047.000000  2047.000000   
        mean      0.148510    10.957173     0.048445     0.313351   
        std       0.355692     1.245863     0.032034     0.121919   
        min       0.000000     8.400000     0.009000     0.000000   
        25%       0.000000     9.900000     0.033000     0.260000   
        50%       0.000000    10.900000     0.040000     0.300000   
        75%       0.000000    12.000000     0.051000     0.370000   
        max       1.000000    14.900000     0.610000     1.660000   
2       count  1978.000000  1978.000000  1978.000000  1978.000000   
        mean      0.028311    10.387713     0.048294     0.337659   
        std       0.165903     1.154493     0.022730     0.125281   
        min       0.000000     8.000000     0.014000     0.000000   
        25%       0.000000     9.400000     0.037000     0.260000   
        50%       0.000000    10.250000     0.044000     0.310000   
        75%       0.000000    11.200000     0.052000     0.390000   
        max       1.000000    14.000000     0.271000     1.000000   
3       count  1372.000000  1372.000000  1372.000000  1372.000000   
        mean      0.901603    10.572558     0.081845     0.270853   
        std       0.297959     1.059402     0.046438     0.196245   
        min       0.000000     8.400000     0.013000     0.000000   
        25%       1.000000     9.700000     0.065000     0.090000   
        50%       1.000000    10.450000     0.076000     0.260000   
        75%       1.000000    11.300000     0.088000     0.430000   
        max       1.000000    14.000000     0.611000     0.760000   

                   density  fixed acidity  free sulfur dioxide           pH  \
Cluster                                                                       
0       count  1100.000000    1100.000000          1100.000000  1100.000000   
        mean      0.996283       6.970182            50.838636     3.181691   
        std       0.002420       0.752114            18.044647     0.140266   
        min       0.988700       4.200000             6.000000     2.850000   
        25%       0.995035       6.400000            40.000000     3.090000   
        50%       0.996290       6.900000            50.250000     3.165000   
        75%       0.998000       7.400000            59.000000     3.250000   
        max       1.010300      10.700000           289.000000     3.740000   
1       count  2047.000000    2047.000000          2047.000000  2047.000000   
        mean      0.993077       6.957108            25.217636     3.207049   
        std       0.002818       1.106360            10.074231     0.159636   
        min       0.987110       3.800000             2.000000     2.740000   
        25%       0.990855       6.200000            18.000000     3.100000   
        50%       0.992600       6.800000            25.000000     3.200000   
        75%       0.994900       7.500000            32.000000     3.310000   
        max       1.003690      15.900000            70.000000     3.850000   
2       count  1978.000000    1978.000000          1978.000000  1978.000000   
        mean      0.994441       6.892644            37.118554     3.194949   
        std       0.002938       0.847187            13.523345     0.151970   
        min  

In [15]:
#the recommender model I will use is positive and negative only
type_1 = [1,1,1,0] #likes first three clusters, but not the fourth
type_2 = [0,0,0,1] #like the fourth one, doesn't like the others
type_3 = [1, 1, 0, 1]
type_4 = [0, 1, 0, 1]

In [16]:
type_1_ratings = []
for row in cluster_data.itertuples():
    type_1_ratings.append(type_1[row[-1]])

In [17]:
type_2_ratings = []
for row in cluster_data.itertuples():
    type_2_ratings.append(type_2[row[-1]])

In [18]:
type_3_ratings = []
for row in cluster_data.itertuples():
    type_3_ratings.append(type_3[row[-1]])

In [19]:
type_4_ratings = []
for row in cluster_data.itertuples():
    type_4_ratings.append(type_4[row[-1]])

In [20]:
ratings = [type_1_ratings, type_2_ratings, type_3_ratings, type_4_ratings]
ratings = np.array(ratings).T
cols = ['type_1_ratings', 'type_2_ratings', 'type_3_ratings', 'type_4_ratings']
ratings_df = pd.DataFrame(ratings, columns=cols)
ratings_df.head()

,type_1_ratings,type_2_ratings,type_3_ratings,type_4_ratings
0,0,1,1,1
1,1,0,1,1
2,0,1,1,1
3,0,1,1,1
4,0,1,1,1


Indices of the clusters

In [21]:
ind = cluster_groups.indices
ind

{0: array([1079, 1081, 1602, ..., 6483, 6484, 6493]),
 1: array([   1,    9,   11, ..., 6494, 6495, 6496]),
 2: array([  14,   15,   86, ..., 6485, 6487, 6490]),
 3: array([   0,    2,    3, ..., 6438, 6466, 6486])}

Create the training set

In [22]:
np.random.seed(107)
num_users_per_type = 1000
num_types = 4
num_ratings_per_cluster = 50 #this will give some users more ratings than others
users = np.zeros([6497, num_users_per_type * num_types])
for n in range(num_types):
    for i in range(num_users_per_type):
        for key, value in ind.items():
            for j in range(num_ratings_per_cluster):
                row = np.random.choice(value)
                users[row, num_users_per_type * n + i] = ratings[row, n]

In [23]:
users

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [24]:
users.sum(axis=0)

array([ 148.,  144.,  148., ...,   96.,   98.,   99.])

In [44]:
users.T.shape

(4000, 6497)

In [43]:
users_sparse = coo_matrix(users.T)

In [45]:
users_sparse.shape

(4000, 6497)

Create the item features matrix. I'm removing the ratings column from the data for test purposes.

In [26]:
#the algorithm I found is much more suited to sparse feature data, but it's what I have right now
item_features = data.values
item_features = np.delete(item_features, -2, axis=1)
item_features[0]

array([  7.4   ,   0.7   ,   0.    ,   1.9   ,   0.076 ,  11.    ,
        34.    ,   0.9978,   3.51  ,   0.56  ,   9.4   ,   1.    ])

In [27]:
item_features = csr_matrix(item_features, dtype=np.float32)

In [46]:
item_features.shape

(6497, 12)

Create test set

In [28]:
np.random.seed(17)
num_users_per_type = 1000
num_types = 4
num_ratings_per_cluster = 50 #this will give some users more ratings than others
test = np.zeros([6497, num_users_per_type * num_types])
for n in range(num_types):
    for i in range(num_users_per_type):
        for key, value in ind.items():
            for j in range(num_ratings_per_cluster):
                row = np.random.choice(value)
                test[row, num_users_per_type * n + i] = ratings[row, n]

In [29]:
test.sum(axis=0)

array([ 149.,  150.,  146., ...,   95.,   98.,  100.])

In [47]:
test.shape

(6497, 4000)

In [48]:
test_sparse = coo_matrix(test.T)

In [49]:
test_sparse.shape

(4000, 6497)

Using the Weighted Approximate-Rank Pairwise loss function in a hybrid recommender system called <a href='http://lyst.github.io/lightfm/docs/home.html'>lightfm.</a>

In [50]:
model = LightFM(loss='warp')

In [51]:
model = model.fit(users_sparse, item_features=item_features)

In [52]:
train_auc = auc_score(model, users_sparse, item_features=item_features).mean()

In [53]:
test_auc = auc_score(model, test_sparse, item_features=item_features).mean()

In [54]:
print(train_auc)
print(test_auc)

0.567562
0.567451


This is terrible...but at least I have something to work with. It kinda makes sense that the AUC's would be so similar, since I made up data.

In [36]:
with open('ratings_types.csv', 'w') as file:
    ratings_df.to_csv(file, index=False)

In [37]:
users_df = pd.DataFrame(users)

In [38]:
with open('users.csv', 'w') as file:
    users_df.to_csv(file, index=False)

In [39]:
test_df = pd.DataFrame(test)

In [40]:
with open('test.csv', 'w') as file:
    test_df.to_csv(file, index=False)

In [41]:
with open('data.csv', 'w') as file:
    data.to_csv(file, index=False)